In [1]:
import pandas as pd
import numpy as np

from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc, precision_recall_fscore_support as score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, StratifiedKFold, cross_val_score

from hyperopt import hp, tpe, fmin, Trials, space_eval, rand, STATUS_OK

from xgboost import XGBClassifier

c:\Users\Convergytics\anaconda3\envs\kamenv\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
## Load the breast cancer dataset
data = datasets.load_breast_cancer()

df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   mean radius              569 non-null    float64
 1   mean texture             569 non-null    float64
 2   mean perimeter           569 non-null    float64
 3   mean area                569 non-null    float64
 4   mean smoothness          569 non-null    float64
 5   mean compactness         569 non-null    float64
 6   mean concavity           569 non-null    float64
 7   mean concave points      569 non-null    float64
 8   mean symmetry            569 non-null    float64
 9   mean fractal dimension   569 non-null    float64
 10  radius error             569 non-null    float64
 11  texture error            569 non-null    float64
 12  perimeter error          569 non-null    float64
 13  area error               569 non-null    float64
 14  smoothness error         5

In [3]:
# Check the target distribution
df['target'].value_counts(normalize=True)

1    0.627417
0    0.372583
Name: target, dtype: float64

In [4]:
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(df.drop('target', axis=1), df['target'], test_size=0.2, random_state=2022)

print(f'The training set has {X_train.shape[0]} rows and {X_train.shape[1]} columns')
print(f'The test set has {X_test.shape[0]} rows and {X_test.shape[1]} columns')

The training set has 455 rows and 30 columns
The test set has 114 rows and 30 columns


Standardization - Rescale and is calculated by extracting the mean and divided by the standard deviation. After standardization, each feature has zero mean and unit standard deviation.

Standardization should be fit on the training dataset only to prevent test dataset information from leaking into the training process.

Then, the test dataset is standardized using the fitting results from the training dataset.

Different types of scalers - StandardScaler, MinMaxScaler, and RobustScaler.

In [5]:
sc = StandardScaler() # Initialize the scaler

# Standardize the training set
X_train_std = sc.fit_transform(X_train)
X_train_transformed = pd.DataFrame(X_train_std, index=X_train.index, columns=X_train.columns)

In [6]:
X_train_transformed.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
544,-0.055955,0.334807,-0.074122,-0.183103,-0.042860,-0.042485,-0.646152,-0.643657,-0.702219,0.604869,...,-0.238758,-0.159261,-0.227195,-0.325798,-0.262873,-0.319339,-0.642079,-0.701544,-1.034791,0.058717
226,-1.051178,-0.913059,-1.046577,-0.927708,0.638402,-0.508435,-1.027508,-0.945635,-0.076080,0.254539,...,-0.994051,-0.992566,-1.015921,-0.864491,0.070490,-0.876590,-1.167024,-1.067218,-0.455101,-0.064280
325,-0.404138,-0.474877,-0.432019,-0.459996,0.459500,-0.519429,-0.708582,-0.712276,-0.377969,-0.431402,...,-0.525470,-0.773718,-0.548516,-0.535617,0.256654,-0.839398,-0.810443,-0.892444,-0.339480,-0.825978
559,-0.740715,1.104007,-0.714724,-0.712087,-0.269708,-0.036798,0.291190,-0.188988,-1.566887,0.431176,...,-0.788645,1.929892,-0.745544,-0.719713,-0.115674,-0.016759,0.420449,-0.270292,-1.251779,0.191090
141,0.593987,-0.296270,0.569841,0.482723,0.059473,0.183097,0.080187,0.292395,0.195992,-0.042801,...,0.803248,-0.071722,0.688279,0.674421,-0.046404,-0.193894,0.029959,0.114733,-0.174760,-0.125503


In [7]:
## Standardize the test set
X_test_transformed = pd.DataFrame(sc.transform(X_test), index=X_test.index, columns=X_test.columns)

In [8]:
## Summary statistics after standardization
X_train_transformed.describe().T

,count,mean,std,min,25%,50%,75%,max
mean radius,455.0,-5.504754e-16,1.001101,-2.054815,-0.687036,-0.230046,0.470672,4.075816
mean texture,455.0,1.247354e-15,1.001101,-2.282378,-0.704684,-0.100993,0.575331,4.759493
mean perimeter,455.0,2.732857e-17,1.001101,-2.005589,-0.693931,-0.224926,0.502630,4.073198
mean area,455.0,4.489693e-16,1.001101,-1.470697,-0.666133,-0.295436,0.328521,5.407849
mean smoothness,455.0,4.567775e-16,1.001101,-3.130723,-0.696213,0.005802,0.645559,3.457912
mean compactness,455.0,-9.857804e-16,1.001101,-1.604877,-0.723306,-0.181815,0.486401,4.575313
mean concavity,455.0,-7.261591e-16,1.001101,-1.111291,-0.755752,-0.338665,0.506229,4.271613
mean concave points,455.0,8.159529e-16,1.001101,-1.264112,-0.737026,-0.405323,0.582980,4.005437
mean symmetry,455.0,-1.581153e-15,1.001101,-2.256385,-0.707810,-0.064899,0.553786,4.090724
mean fractal dimension,455.0,2.541557e-15,1.001101,-1.885716,-0.719174,-0.138479,0.492999,5.103231


We can see that after using StandardScaler, all the features have zero mean and unit standard deviation.

So, before standardization the mean and standard deviation can be very different in scale.

In [9]:
X_train.describe().T

,count,mean,std,min,25%,50%,75%,max
mean radius,455.0,14.062846,3.450257,6.981000,11.695000,13.270000,15.685000,28.11000
mean texture,455.0,19.294088,4.203790,9.710000,16.335000,18.870000,21.710000,39.28000
mean perimeter,455.0,91.534527,23.831936,43.790000,75.015000,86.180000,103.500000,188.50000
mean area,455.0,647.555385,343.109583,143.500000,419.250000,546.300000,760.150000,2501.00000
mean smoothness,455.0,0.096379,0.013989,0.052630,0.086650,0.096460,0.105400,0.14470
mean compactness,455.0,0.104041,0.052810,0.019380,0.065885,0.094450,0.129700,0.34540
mean concavity,455.0,0.088112,0.079375,0.000000,0.028190,0.061260,0.128250,0.42680
mean concave points,455.0,0.048266,0.038224,0.000000,0.020125,0.032790,0.070525,0.20120
mean symmetry,455.0,0.180841,0.026861,0.120300,0.161850,0.179100,0.195700,0.29060
mean fractal dimension,455.0,0.062771,0.006801,0.049960,0.057885,0.061830,0.066120,0.09744


XGB Base Model with default hyperparameters

In [11]:
pd.set_option('display.max_columns', None)

In [12]:
xgboost = XGBClassifier()

xgboost.get_params()

{'objective': 'binary:logistic',
 'use_label_encoder': True,
 'base_score': None,
 'booster': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': None,
 'enable_categorical': False,
 'gamma': None,
 'gpu_id': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': None,
 'max_delta_step': None,
 'max_depth': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'n_estimators': 100,
 'n_jobs': None,
 'num_parallel_tree': None,
 'predictor': None,
 'random_state': None,
 'reg_alpha': None,
 'reg_lambda': None,
 'scale_pos_weight': None,
 'subsample': None,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None}

In [13]:
xgboost = XGBClassifier(seed=2022).fit(X_train_transformed, y_train)

xgboost_score = xgboost.score(X_test_transformed, y_test)

xgboost_predictions = xgboost.predict(X_test_transformed)

xgboost_predct_proba = xgboost.predict_proba(X_test_transformed)[:,1]

[16:37:26] WARNING: D:\bld\xgboost-split_1645118015404\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


c:\Users\Convergytics\anaconda3\envs\kamenv\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\Users\Convergytics\anaconda3\envs\kamenv\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
c:\Users\Convergytics\anaconda3\envs\kamenv\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype

We want to capture as many cancer patients as possible for this dataset, so we will use recall as the performance metric to optimize.

In [15]:
xgboost_score

0.956140350877193

In [16]:
xgboost_predictions

array([1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1,
       1, 1, 0, 1])

In [17]:
confusion_matrix(y_test, xgboost_predictions)

array([[44,  2],
       [ 3, 65]], dtype=int64)

In [19]:
print(classification_report(y_test, xgboost_predictions))

              precision    recall  f1-score   support

           0       0.94      0.96      0.95        46
           1       0.97      0.96      0.96        68

    accuracy                           0.96       114
   macro avg       0.95      0.96      0.95       114
weighted avg       0.96      0.96      0.96       114



In [14]:
## Get performance metrics
precision, recall, fscore, support = score(y_test, xgboost_predictions)

print(f'The recall value for the baseline XGBoost model is {recall[1]:.3f}')

The recall value for the baseline XGBoost model is 0.956


Grid Search

It is an exhaustive hyperparameter search method. It trains models for every combination of specified hyperparameter values. It can take a long time to run if we test out more hyperparameters and values.

So we would like to keep the grid search space relatively small so the process can finish in a reasonable timeframe.

We have 3 hyperparameters for this example:
- colsample_bytree - percentage of columns to be randomly sampled for each tree.
- reg_alpha - provides l1 regularization to the weight, higher values result in more conservative models.
- reg_lambda - provides l2 regularization to the weight, higher values result in more conservative models.

Scoring is the metric to evaluate the cross-validation results for each model. Since recall is the evaluation metric for the model, we set scoring = ['recall']. It can take more than one metric in the list.

StratifiedKFold is used for the cross-validation. It helps us keep the class ratio in the folds the same as the training dataset. 

- n_splits = 3 means we are doing 3-fold cross-validation.
- shuffle = True means the data is shuffled before splitting.
- random_state = 2022 makes the shuffle reproducible.

In [20]:
## Define the grid search space
param_grid = {
    # Percentage of features to be randomly selected for each tree
    'colsample_bytree': [0.3, 0.5, 0.8, 1.0],
    'reg_alpha': [0, 0.5, 1.0, 5.0],
    'reg_lambda': [0, 0.5, 1.0, 5.0]
}

In [21]:
## Setup scoring metrics
scoring = ['recall']

# Setup the k-fold cross-validation
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=2022)

We specified a few options for GridSearchCV.

- estimator=xgboost, means we are using XGBoost as the model.
- param_grid=param_grid
- scoring=scoring
- refit='recall', enables refitting the model with the best parameters on the whole training dataset
- n_jobs=-1, means parallel processing using all the processors.
- cv=kfold
- verbose, controls the number of messages returned by grid search. verbose=0 means silent output.

After fitting GridSearchCV on the training dataset, we will have 64 hyperparameter combinations. Since 3-fold cv is used, there are 192 (64*3) models trained in total.

In [22]:
## Define the grid search
grid_search = GridSearchCV(estimator=xgboost, param_grid=param_grid, scoring=scoring, refit='recall', cv=kfold, n_jobs=-1, verbose=1)

In [23]:
## Fit the grid search
grid_result = grid_search.fit(X_train_transformed, y_train)

Fitting 3 folds for each of 64 candidates, totalling 192 fits
[17:00:43] WARNING: D:\bld\xgboost-split_1645118015404\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


c:\Users\Convergytics\anaconda3\envs\kamenv\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\Users\Convergytics\anaconda3\envs\kamenv\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [24]:
# Print grid search results
grid_result

GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=2022, shuffle=True),
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1,
                                     enable_categorical=False, gamma=0,
                                     gpu_id=-1, importance_type=None,
                                     interaction_constraints='',
                                     learning_rate=0.300000012,
                                     max_delta_step=0, max_depth=6,
                                     min_child_weigh...
                                     n_estimators=100, n_jobs=8,
                                     num_parallel_tree=1, predictor='auto',
                                     random_state=2022, reg_alpha=0,
                                     reg_lambda=1, scale_pos_weight=1,
                                     see

In [25]:
## Print the best score and the best parameters
print(f'The best score is {grid_result.best_score_:.3f}')
print(f'The best score standard deviation is {grid_result.cv_results_["std_test_recall"][grid_result.best_index_]:.3f}')
print(f'The best hyperparameters are {grid_result.best_params_}')

The best score is 0.990
The best score standard deviation is 0.008
The best hyperparameters are {'colsample_bytree': 0.5, 'reg_alpha': 0, 'reg_lambda': 0}


The grid search cross-validation results show that 50% of features and no l1&l2 regularization gave us the best results. The best recall is 99% and the standard deviation of the score is 0.8%.

In [26]:
## Make predictions using the best model
grid_predict = grid_result.predict(X_test_transformed)

## Get predictions probabilities
grid_predict_prob = grid_result.predict_proba(X_test_transformed)[:,1]

c:\Users\Convergytics\anaconda3\envs\kamenv\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [27]:
## Get the confusion matrix
confusion_matrix(y_test, grid_predict)

array([[43,  3],
       [ 2, 66]], dtype=int64)

In [28]:
print(classification_report(y_test, grid_predict))

              precision    recall  f1-score   support

           0       0.96      0.93      0.95        46
           1       0.96      0.97      0.96        68

    accuracy                           0.96       114
   macro avg       0.96      0.95      0.95       114
weighted avg       0.96      0.96      0.96       114



In [29]:
## Get performance metrics
precision, recall, fscore, support = score(y_test, grid_predict)

print(f'The recall value for the grid search XGBoost model is {recall[1]:.3f}')

The recall value for the grid search XGBoost model is 0.971


We can see that the grid search recall value is better than the baseline xgboost model.

Random Search

It randomly picks a fixed number of hyperparameter combinations, we can afford to try more.

- learning_rate, shrinks the weights to make the boosting process more conservative.
- max_depth, max depth of the tree. Increasing it increases the model complexity.
- gamma, specifies the min loss reduction required to do a plit.

If at least one of the parameters is a distribution, sampling with replacement is used for a random search. If all parameters are provided as a list, sampling without replacement is used. Each list is treated as a uniform distribution.

In [30]:
## Define the search space
param_grid_random = {
    'learning_rate': [0.0001, 0.001, 0.01, 0.1, 0.9],
    'max_depth': range(3,21,3),
    'gamma': [i/10.0 for i in range(0,5)],
    'colsample_bytree': [i/10.0 for i in range(3,10)],
    'reg_alpha': [1e-5, 1e-2, 0.1, 1, 10, 100],
    'reg_lambda': [1e-5, 1e-2, 0.1, 1, 10, 100],
    'n_estimators': range(100,1001,100)
}

scoring = ['recall']

kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=2022)

For random search, we need to specify a value for n_iter, the number of parameter combinations sampled. So we are randomly testing 64 combinations for this case.

In [31]:
## Define random search
random_search = RandomizedSearchCV(estimator=xgboost, 
                                   param_distributions=param_grid_random, 
                                   scoring=scoring, 
                                   refit='recall', 
                                   cv=kfold, 
                                   n_iter=64, 
                                   n_jobs=-1, 
                                   verbose=0)

# Fit the random search
random_result = random_search.fit(X_train_transformed, y_train)

random_result

c:\Users\Convergytics\anaconda3\envs\kamenv\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\Users\Convergytics\anaconda3\envs\kamenv\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[18:26:50] WARNING: D:\bld\xgboost-split_1645118015404\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


RandomizedSearchCV(cv=StratifiedKFold(n_splits=3, random_state=2022, shuffle=True),
                   estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                           colsample_bylevel=1,
                                           colsample_bynode=1,
                                           colsample_bytree=1,
                                           enable_categorical=False, gamma=0,
                                           gpu_id=-1, importance_type=None,
                                           interaction_constraints='',
                                           learning_rate=0.300000012,
                                           max_delta_step=0, max_depth=6,
                                           min_child...
                                           verbosity=None),
                   n_iter=64, n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.3, 0.4, 0.5, 0.6,
                                          

In [32]:
print(f'The best score is {random_result.best_score_:.3f}')
print(f'The best score standard deviation is {random_result.cv_results_["std_test_recall"][random_result.best_index_]:.3f}')
print(f'The best hyperparameters are {random_result.best_params_}')

The best score is 0.993
The best score standard deviation is 0.005
The best hyperparameters are {'reg_lambda': 0.01, 'reg_alpha': 1e-05, 'n_estimators': 1000, 'max_depth': 15, 'learning_rate': 0.01, 'gamma': 0.2, 'colsample_bytree': 0.4}


In [33]:
random_predictions = random_result.predict(X_test_transformed)

random_predictions_prob = random_result.predict_proba(X_test_transformed)[:,1]

precision, recall, fscore, support = score(y_test, random_predictions)

print(f'The recall value for the random search XGBoost model is {recall[1]:.3f}')

The recall value for the random search XGBoost model is 0.985


c:\Users\Convergytics\anaconda3\envs\kamenv\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


The random search recall value on the test dataset is creased from 97.1% to 98.5%.

Bayesian Optimization

We defined an objective function that takes in the parameters and returns the loss. Since the goal is to maximize the recall value, we set max(scores) as the best_score, and set the loss to be -best_score. This setting ensures maximizing recall while minimizing the loss.

fmin is used to optimize the objective function. Hyperopt currently has three algorithms. They are random search, Tree of Parzen Estimators(TPE) and adaptive TPE. We are using TPE as the search algorithm.

In [34]:
# Space
space = {
    'learning_rate': hp.choice('learning_rate', [0.0001, 0.001, 0.01, 0.1, 1.0]),
    'max_depth': hp.choice('max_depth', range(3,21,3)),
    'gamma': hp.choice('gamma', [i/10.0 for i in range(0,5)]),
    'colsample_bytree': hp.choice('colsample_bytree', [i/10.0 for i in range(3,10)]),
    'reg_alpha': hp.choice('reg_alpha', [1e-5, 1e-2, 0.1, 1, 10, 100]),
    'reg_lambda': hp.choice('reg_lambda', [1e-5, 1e-2, 0.1, 1, 10, 100]),
}

kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=2022)

def objective(params):
    xgboost = XGBClassifier(seed=2022, **params)
    scores = cross_val_score(xgboost, X_train_transformed, y_train, scoring='recall', cv=kfold, n_jobs=-1)
    
    # extract the best score
    best_score = max(scores)
    
    # loss must be minimized
    loss = - best_score
    
    # dictionary with information for evaluation
    return {'loss': loss, 'status': STATUS_OK}

# Trials object to store all information about the optimization
bayes_trials = Trials()

# Optimize
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=64, trials=bayes_trials)

100%|██████████| 64/64 [00:14<00:00,  4.37trial/s, best loss: -1.0]               


def objective(params):
    cv_result = cross_val_score(XGBClassifier(seed=2022,**params), X_train_transformed, y_train, cv=kfold, scoring='recall')
    return {'loss': 1 - cv_result.mean(), 'status': STATUS_OK}

After the bayesian optimization search, we get the best loss of -1.0, meaning that the recall value is close to 100%.

In [35]:
## Print the index of the best hyperparameter
print(best)

# Print the values of the best hyperparameters
print(space_eval(space, best))

{'colsample_bytree': 1, 'gamma': 3, 'learning_rate': 4, 'max_depth': 1, 'reg_alpha': 2, 'reg_lambda': 0}
{'colsample_bytree': 0.4, 'gamma': 0.3, 'learning_rate': 1.0, 'max_depth': 6, 'reg_alpha': 0.1, 'reg_lambda': 1e-05}


In [36]:
## Train model using the best hyperparameters
xgboost_best = XGBClassifier(seed=2022,
                             colsample_bytree=space_eval(space, best)['colsample_bytree'],
                             gamma=space_eval(space, best)['gamma'],
                             learning_rate=space_eval(space, best)['learning_rate'],
                             max_depth=space_eval(space, best)['max_depth'],
                             reg_alpha=space_eval(space, best)['reg_alpha'],
                             reg_lambda=space_eval(space, best)['reg_lambda'],
).fit(X_train_transformed, y_train)

[18:50:16] WARNING: D:\bld\xgboost-split_1645118015404\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


c:\Users\Convergytics\anaconda3\envs\kamenv\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\Users\Convergytics\anaconda3\envs\kamenv\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [37]:
# Make predictions using the best model
bayesian_opt_predictions = xgboost_best.predict(X_test_transformed)

# Get predicted probabilities
bayesian_opt_predictions_prob = xgboost_best.predict_proba(X_test_transformed)[:,1]

# Get performance metrics
precision, recall, fscore, support = score(y_test, bayesian_opt_predictions)

# Print result
print(f'The recall value for the bayesian optimization XGBoost model is {recall[1]:.3f}')

The recall value for the bayesian optimization XGBoost model is 0.985


c:\Users\Convergytics\anaconda3\envs\kamenv\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


The recall value on the test dataset is 98.5%, the same as the random search result.